シンプルなニューラルネットワークの確認をする
（テキストＰ１１０～）

In [1]:
#最初のお約束の設定
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
#import common.gradient as g #例えば、本書の共通ライブラリを読み込む場合の設定
import numpy as np
import matplotlib.pylab as plt
######################################################

In [2]:
#今回、追加で読み込むもの
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

シンプルなネットワークを実装する前に、シンプルなネットワークで使われているnumpyの使い方を２〜３確認しておく。
●np.random.randn・・・標準正規分布(平均0、分散1（標準偏差1））)に従った乱数を得る。引数は得たい乱数列のshape
https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html?highlight=random%20randn

In [3]:
#np.random.randn(2,3)　について得られる乱数は(2,3)行列
r = np.random.randn(2,3)
print(r.shape)
print(r) #実行する毎に内容が異なることに注意（乱数だから当たり前）

(2, 3)
[[-0.58973736  0.08235795  0.81268808]
 [ 0.88913709 -0.04516613  0.95275348]]


いよいよ、シンプルネットワークを実装する。なお、以下の実装は、テキストP110をそのまま写経したもの

In [4]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) #重みを乱数で初期化する。乱数で初期化するコト自体は非常に重要。理由はテキストに記載があるので参照。

    def predict(self, x):
        return np.dot(x, self.W)
    
    #損失関数で損失を計算。xがニューラルネットの予測、tが正解データ(one hot 表現)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        
        return loss

実際にシンプルネットの動作を確かめる。

In [5]:
net = simpleNet()
print(net.W) #重みを表示してみる（ランダム初期化）

[[-1.95293667  1.27081936  0.82490609]
 [ 1.0175071   0.31572356 -0.60563498]]


In [6]:
x = np.array([0.6,0.9]) #サンプルの入力データ
p = net.predict(x)
print(p)

[-0.25600561  1.04664282 -0.05012783]


In [7]:
t = np.array([0,0,1]) #正解ラベル
net.loss(x, t)

1.570366551130616

In [8]:
#テキストの実行結果と合っているかを確認するため、重みをテキストと合わせる
net.W = np.array([[0.47355232, 0.9977393, 0.84668094],
                  [0.85557411, 0.03563661, 0.69422093]])

In [9]:
print(net.W)

[[0.47355232 0.9977393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]


In [11]:
x = np.array([0.6,0.9]) #サンプルの入力データ
p = net.predict(x)
print(p)
print(np.argmax(p))
t = np.array([0,0,1]) #正解ラベル
net.loss(x, t)

[1.05414809 0.63071653 1.1328074 ]
2


0.9280682857864075

結果はテキストと同じになっており、実装はただしそうである。

以下、勾配を求めてみる。勾配を求める対象の関数は損失関数になる。
また、numeriacl_gradientの仕様は関数を渡すことになるので、以下のように関数を定義する必要がある。
なお、tはグローバル定義になる（上記に定義済み)

In [15]:
def f(W):
    print("====W=====")
    print(W)
    print(x)
    print(t)
    loss = net.loss(x,t)
    print(loss)
    print("==========")
    return loss

print("== start of W === ")
print(net.W)
print("=================")
dW = numerical_gradient(f, net.W)
print(dW)

== start of W === 
[[0.47355232 0.9977393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]
====W=====
[[0.47365232 0.9977393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9280902109609267
====W=====
[[0.47345232 0.9977393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9280463614466788
====W=====
[[0.47355232 0.9978393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.928082642357028
====W=====
[[0.47355232 0.9976393  0.84668094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9280539298710598
====W=====
[[0.47355232 0.9977393  0.84678094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9280320052165812
====W=====
[[0.47355232 0.9977393  0.84658094]
 [0.85557411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9281045672167824
====W=====
[[0.47355232 0.9977393  0.84668094]
 [0.85567411 0.03563661 0.69422093]]
[0.6 0.9]
[0 0 1]
0.9281011738612368
====W=====
[[0.47355232 0.9977393  0.84668094]
 [0.85547411 0.

上記の出力結果を見ると、関数fに0.0001をプラスマイナスしながら引数が渡されて実行されていることがわかる。
なお、numerical_gradient関数にnet.Wを渡すと、net.Wへのポインタを受け取ったnumerical_gradientの内部処理がnet.Wの値を微小(0.0001)だけnet.Wの値を直接変えながら、net.loss(x,t)を実行することに注意するべし。